In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.over_sampling import smote
import sklearn.metrics as mt
from imblearn.combine import smoteTomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from imblearn.combine import smoteENN
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# record start time
start = time.time()
le = LabelEncoder()
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/insurance.csv')


X=dataset.drop(['labels'],axis=1)
y=dataset["labels"]
#y = le.fit_transform(y)
#imblea#n smoteenn
#smoteenn_enn = smoteenn()
smoteenn_enn = smoteennENN()
X, y = smoteenn_enn.fit_resample(X, y)
#smoteenn_enn = smoteennTomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smoteenn_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)

end = time.time()

print("The time of execution of preprocess:",
      (end-start), "s")

The time of execution of preprocess: 54.88401007652283 s


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-insurance-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-insurance-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.943789 using {'class_weight': {0: 100, 1: 50}}
0.943252 (0.002992) with: {'class_weight': {0: 100, 1: 10}}
0.940591 (0.002603) with: {'class_weight': {0: 100, 1: 20}}
0.942038 (0.002477) with: {'class_weight': {0: 100, 1: 30}}
0.942963 (0.001590) with: {'class_weight': {0: 100, 1: 40}}
0.943789 (0.002510) with: {'class_weight': {0: 100, 1: 50}}
0.942740 (0.002396) with: {'class_weight': {0: 100, 1: 60}}
0.942740 (0.002302) with: {'class_weight': {0: 100, 1: 70}}
0.942567 (0.002231) with: {'class_weight': {0: 100, 1: 80}}
0.942754 (0.003085) with: {'class_weight': {0: 100, 1: 90}}
0.943344 (0.002671) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-insurance-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-insurance-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.944360 using {'class_weight': {0: 100, 1: 50}}
0.943725 (0.002624) with: {'class_weight': {0: 100, 1: 10}}
0.941176 (0.002665) with: {'class_weight': {0: 100, 1: 20}}
0.942534 (0.002374) with: {'class_weight': {0: 100, 1: 30}}
0.943184 (0.001621) with: {'class_weight': {0: 100, 1: 40}}
0.944360 (0.002246) with: {'class_weight': {0: 100, 1: 50}}
0.942903 (0.002613) with: {'class_weight': {0: 100, 1: 60}}
0.942665 (0.002259) with: {'class_weight': {0: 100, 1: 70}}
0.942937 (0.002461) with: {'class_weight': {0: 100, 1: 80}}
0.942870 (0.002909) with: {'class_weight': {0: 100, 1: 90}}
0.943612 (0.002222) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-insurance-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-insurance-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder


Best: 0.943853 using {'class_weight': {0: 100, 1: 50}}
0.942853 (0.002590) with: {'class_weight': {0: 100, 1: 10}}
0.940545 (0.002149) with: {'class_weight': {0: 100, 1: 20}}
0.941765 (0.002458) with: {'class_weight': {0: 100, 1: 30}}
0.942517 (0.001529) with: {'class_weight': {0: 100, 1: 40}}
0.943853 (0.002253) with: {'class_weight': {0: 100, 1: 50}}
0.942741 (0.003049) with: {'class_weight': {0: 100, 1: 60}}
0.942358 (0.002312) with: {'class_weight': {0: 100, 1: 70}}
0.942442 (0.002700) with: {'class_weight': {0: 100, 1: 80}}
0.942995 (0.003198) with: {'class_weight': {0: 100, 1: 90}}
0.943527 (0.002686) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-insurance-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-insurance-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.949417 using {'class_weight': {0: 100, 1: 100}}
0.948688 (0.001727) with: {'class_weight': {0: 100, 1: 10}}
0.949004 (0.001715) with: {'class_weight': {0: 100, 1: 20}}
0.949160 (0.001706) with: {'class_weight': {0: 100, 1: 30}}
0.949255 (0.001698) with: {'class_weight': {0: 100, 1: 40}}
0.949313 (0.001693) with: {'class_weight': {0: 100, 1: 50}}
0.949351 (0.001690) with: {'class_weight': {0: 100, 1: 60}}
0.949380 (0.001688) with: {'class_weight': {0: 100, 1: 70}}
0.949398 (0.001686) with: {'class_weight': {0: 100, 1: 80}}
0.949411 (0.001683) with: {'class_weight': {0: 100, 1: 90}}
0.949417 (0.001679) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-insurance-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-insurance-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.884028 using {'class_weight': {0: 100, 1: 100}}
0.731398 (0.003691) with: {'class_weight': {0: 100, 1: 10}}
0.808812 (0.002632) with: {'class_weight': {0: 100, 1: 20}}
0.840947 (0.001931) with: {'class_weight': {0: 100, 1: 30}}
0.857945 (0.002088) with: {'class_weight': {0: 100, 1: 40}}
0.867711 (0.001639) with: {'class_weight': {0: 100, 1: 50}}
0.874116 (0.001927) with: {'class_weight': {0: 100, 1: 60}}
0.878526 (0.002764) with: {'class_weight': {0: 100, 1: 70}}
0.882285 (0.003078) with: {'class_weight': {0: 100, 1: 80}}
0.883986 (0.003402) with: {'class_weight': {0: 100, 1: 90}}
0.884028 (0.003683) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-insurance-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-insurance-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.883946 using {'class_weight': {0: 100, 1: 90}}
0.724364 (0.004198) with: {'class_weight': {0: 100, 1: 10}}
0.810276 (0.002743) with: {'class_weight': {0: 100, 1: 20}}
0.844220 (0.001951) with: {'class_weight': {0: 100, 1: 30}}
0.861409 (0.002092) with: {'class_weight': {0: 100, 1: 40}}
0.870777 (0.001639) with: {'class_weight': {0: 100, 1: 50}}
0.876553 (0.001920) with: {'class_weight': {0: 100, 1: 60}}
0.880206 (0.002760) with: {'class_weight': {0: 100, 1: 70}}
0.883086 (0.003098) with: {'class_weight': {0: 100, 1: 80}}
0.883946 (0.003448) with: {'class_weight': {0: 100, 1: 90}}
0.883135 (0.003749) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-insurance-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-insurance-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.944224 using {'class_weight': {0: 100, 1: 100}}
0.926166 (0.001736) with: {'class_weight': {0: 10, 1: 100}}
0.932841 (0.001772) with: {'class_weight': {0: 20, 1: 100}}
0.936174 (0.001779) with: {'class_weight': {0: 30, 1: 100}}
0.938495 (0.001732) with: {'class_weight': {0: 40, 1: 100}}
0.940248 (0.001726) with: {'class_weight': {0: 50, 1: 100}}
0.941489 (0.001730) with: {'class_weight': {0: 60, 1: 100}}
0.942392 (0.001684) with: {'class_weight': {0: 70, 1: 100}}
0.943112 (0.001677) with: {'class_weight': {0: 80, 1: 100}}
0.943751 (0.001685) with: {'class_weight': {0: 90, 1: 100}}
0.944224 (0.001728) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-insurance-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-insurance-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.884830 using {'class_weight': {0: 100, 1: 100}}
0.783497 (0.004218) with: {'class_weight': {0: 10, 1: 100}}
0.829738 (0.003742) with: {'class_weight': {0: 20, 1: 100}}
0.850816 (0.003220) with: {'class_weight': {0: 30, 1: 100}}
0.863649 (0.003137) with: {'class_weight': {0: 40, 1: 100}}
0.872124 (0.002422) with: {'class_weight': {0: 50, 1: 100}}
0.877014 (0.002177) with: {'class_weight': {0: 60, 1: 100}}
0.880404 (0.002262) with: {'class_weight': {0: 70, 1: 100}}
0.882421 (0.002026) with: {'class_weight': {0: 80, 1: 100}}
0.883966 (0.001780) with: {'class_weight': {0: 90, 1: 100}}
0.884830 (0.001708) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-insurance-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-insurance-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder